In [1]:
import os

directory_path = '/Users/mamduhzabidi/Pictures/Photos Library.photoslibrary/originals/0/'

files = [file for file in os.listdir(directory_path) if file.lower().endswith('.jpeg')]

image_path = os.path.join(directory_path, files[0])

image_path

'/Users/mamduhzabidi/Pictures/Photos Library.photoslibrary/originals/0/0E62EDEE-1691-43AA-BED9-C97B7FA2F733.jpeg'

In [ ]:
from sturdy_barnacle.image_processor import ImageProcessor

image_processor=ImageProcessor(image_path=image_path)

/Users/mamduhzabidi/Documents/GitHub/sturdy-barnacle/myenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

In [ ]:
image_processor.describe_image()

In [12]:
import sturdy_barnacle.image_processor

sturdy_barnacle.image_processor.__file__

'/Users/mamduhzabidi/Documents/GitHub/sturdy-barnacle/sturdy_barnacle/image_processor.py'